# 🚀 Dual Hybrid Model: Swin-Base + MobileViT

## Architecture
- **Swin-Base**: Larger Swin Transformer (1024-dim features)
- **MobileViT**: Pre-trained on Kvasir (512-dim features)
- **Fusion**: Concatenate → 1536-dim → Reduce to 256-dim
- **Classifier**: CatBoost (GPU optimized)

**Expected Performance**: ~93-94% accuracy (better than Swin-Small)

In [ ]:
# Install packages
!pip install -q timm optuna catboost scikit-learn
print("✅ Packages installed")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
weights_path = '/content/drive/MyDrive/TripleHybridModel/mobilevit_kvasir_v2_best_optuna.pth'

if os.path.exists(weights_path):
    print(f"✅ Found MobileViT weights")
else:
    print(f"❌ Weights not found. Please upload to: MyDrive/TripleHybridModel/")
    weights_path = None

In [ ]:
# Imports
import torch
import torch.nn as nn
import torchvision.models as models
import timm
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, log_loss
from sklearn.preprocessing import label_binarize
import optuna
from catboost import CatBoostClassifier
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎮 Device: {device}")

In [ ]:
# Download dataset
import zipfile, urllib.request, ssl

if not os.path.exists('kvasir-dataset-v2'):
    print("📥 Downloading Kvasir dataset...")
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    url = "https://datasets.simula.no/downloads/kvasir/kvasir-dataset-v2.zip"
    with urllib.request.urlopen(url, context=ctx) as response, open('kvasir-dataset-v2.zip', 'wb') as out:
        out.write(response.read())
    with zipfile.ZipFile('kvasir-dataset-v2.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
    print("✅ Dataset ready!")

## 🏗️ Dual Hybrid Model: Swin-Base + MobileViT

In [ ]:
class DualHybridSwinBase(nn.Module):
    """Swin-Base + MobileViT Hybrid Model"""
    
    def __init__(self, mobilevit_weights_path=None):
        super().__init__()
        
        # 1. Swin-Base (larger than Swin-Small)
        print("Loading Swin-Base...")
        self.swin = models.swin_b(weights=models.Swin_B_Weights.IMAGENET1K_V1)
        self.swin.head = nn.Identity()
        self.swin_dim = 1024  # Swin-Base output
        
        # 2. MobileViT
        print("Loading MobileViT...")
        self.mobilevit = timm.create_model('mobilevitv2_100', pretrained=False, num_classes=8)
        if mobilevit_weights_path and os.path.exists(mobilevit_weights_path):
            self.mobilevit.load_state_dict(torch.load(mobilevit_weights_path, map_location='cpu'))
            print("✅ MobileViT weights loaded")
        self.mobilevit.head.fc = nn.Identity()
        self.mobilevit_dim = 512
        
        # 3. Freeze backbones
        for param in self.swin.parameters():
            param.requires_grad = False
        for param in self.mobilevit.parameters():
            param.requires_grad = False
        
        # 4. Fusion layer (1024 + 512 = 1536 → 256)
        combined_dim = self.swin_dim + self.mobilevit_dim  # 1536
        self.fusion = nn.Sequential(
            nn.Linear(combined_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256)
        )
        
        print(f"✅ Hybrid Model Ready:")
        print(f"   - Swin-Base: {self.swin_dim}-dim")
        print(f"   - MobileViT: {self.mobilevit_dim}-dim")
        print(f"   - Combined: {combined_dim}-dim → Fused: 256-dim")
    
    def forward(self, x):
        with torch.no_grad():
            swin_features = self.swin(x)
            mobilevit_features = self.mobilevit(x)
            combined = torch.cat([swin_features, mobilevit_features], dim=1)
        return self.fusion(combined)

print("✅ Model class defined")

In [ ]:
# Dataset
class KvasirDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.samples = []
        classes = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
        self.class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
        for cls in classes:
            for img in (self.root_dir / cls).glob('*.jpg'):
                self.samples.append((img, self.class_to_idx[cls]))
    
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        img, label = self.samples[idx]
        return self.transform(Image.open(img).convert('RGB')), label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = KvasirDataset('kvasir-dataset-v2', transform=transform)
idx = list(range(len(dataset)))
labels = [s[1] for s in dataset.samples]
tr_idx, temp = train_test_split(idx, test_size=0.3, stratify=labels, random_state=42)
val_idx, te_idx = train_test_split(temp, test_size=0.5, stratify=[labels[i] for i in temp], random_state=42)

print(f"📊 Dataset: Train={len(tr_idx)}, Val={len(val_idx)}, Test={len(te_idx)}")

In [ ]:
# Extract features
model = DualHybridSwinBase(weights_path).to(device).eval()

def extract_features(indices, set_name):
    loader = DataLoader(torch.utils.data.Subset(dataset, indices), batch_size=32, num_workers=2)
    feats, lbls = [], []
    start_time = time.time()
    for img, lbl in tqdm(loader, desc=f"Extracting {set_name}"):
        feats.append(model(img.to(device)).cpu().detach().numpy())
        lbls.append(lbl.numpy())
    extraction_time = time.time() - start_time
    return np.vstack(feats), np.concatenate(lbls), extraction_time

print("🔬 Extracting features from Swin-Base + MobileViT...")
X_train, y_train, train_time = extract_features(tr_idx, "Train")
X_val, y_val, val_time = extract_features(val_idx, "Val")
X_test, y_test, test_time = extract_features(te_idx, "Test")

print(f"\n✅ Features: {X_train.shape}")
print(f"⏱️ Extraction time: Train={train_time:.1f}s, Val={val_time:.1f}s, Test={test_time:.1f}s")

In [ ]:
# Train CatBoost
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    params = {
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'task_type': 'GPU',
        'verbose': False,
        'random_seed': 42
    }
    clf = CatBoostClassifier(**params)
    clf.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=False)
    return accuracy_score(y_val, clf.predict(X_val))

print("🔍 Optimizing CatBoost (50 trials)...")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

print(f"\n🏆 Best Val Acc: {study.best_value*100:.2f}%")
print(f"🔧 Best Params: {study.best_params}")

# Train final model
final_params = study.best_params
final_params.update({'task_type': 'GPU', 'random_seed': 42})
catboost_model = CatBoostClassifier(**final_params)
catboost_model.fit(X_train, y_train)

print("✅ Model trained!")

## 📊 Calculate All Metrics

In [ ]:
def calculate_all_metrics(model, X, y, set_name):
    y_pred = model.predict(X)
    y_pred_proba = model.predict_proba(X)
    
    acc = accuracy_score(y, y_pred)
    prec = precision_score(y, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y, y_pred, average='weighted', zero_division=0)
    
    y_bin = label_binarize(y, classes=range(8))
    auroc = roc_auc_score(y_bin, y_pred_proba, average='weighted', multi_class='ovr')
    loss = log_loss(y, y_pred_proba)
    
    cm = confusion_matrix(y, y_pred)
    tpr_list, fpr_list = [], []
    for i in range(8):
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp
        fp = cm[:, i].sum() - tp
        tn = cm.sum() - tp - fn - fp
        tpr_list.append(tp / (tp + fn) if (tp + fn) > 0 else 0)
        fpr_list.append(fp / (fp + tn) if (fp + tn) > 0 else 0)
    
    print(f"\n{set_name} Metrics:")
    print(f"  Accuracy: {acc*100:.2f}%")
    print(f"  Precision: {prec*100:.2f}%")
    print(f"  Recall: {rec*100:.2f}%")
    print(f"  F1-Score: {f1*100:.2f}%")
    print(f"  AUROC: {auroc*100:.2f}%")
    print(f"  TPR: {np.mean(tpr_list)*100:.2f}%")
    print(f"  FPR: {np.mean(fpr_list)*100:.2f}%")
    print(f"  Loss: {loss:.4f}")
    
    return {'acc': acc, 'prec': prec, 'rec': rec, 'f1': f1, 'auroc': auroc, 
            'tpr': np.mean(tpr_list), 'fpr': np.mean(fpr_list), 'loss': loss}

train_m = calculate_all_metrics(catboost_model, X_train, y_train, "Training")
val_m = calculate_all_metrics(catboost_model, X_val, y_val, "Validation")
test_m = calculate_all_metrics(catboost_model, X_test, y_test, "Test")

In [ ]:
# Summary Table
print("\n" + "="*80)
print("SWIN-BASE + MOBILEVIT - COMPLETE RESULTS")
print("="*80)
print(f"\n{'Metric':<15} | {'Training':<12} | {'Validation':<12} | {'Test':<12}")
print("-" * 65)
print(f"{'Accuracy':<15} | {train_m['acc']*100:<12.2f} | {val_m['acc']*100:<12.2f} | {test_m['acc']*100:<12.2f}")
print(f"{'Precision':<15} | {train_m['prec']*100:<12.2f} | {val_m['prec']*100:<12.2f} | {test_m['prec']*100:<12.2f}")
print(f"{'Recall (TPR)':<15} | {train_m['rec']*100:<12.2f} | {val_m['rec']*100:<12.2f} | {test_m['rec']*100:<12.2f}")
print(f"{'F1-Score':<15} | {train_m['f1']*100:<12.2f} | {val_m['f1']*100:<12.2f} | {test_m['f1']*100:<12.2f}")
print(f"{'AUROC':<15} | {train_m['auroc']*100:<12.2f} | {val_m['auroc']*100:<12.2f} | {test_m['auroc']*100:<12.2f}")
print(f"{'TPR':<15} | {train_m['tpr']*100:<12.2f} | {val_m['tpr']*100:<12.2f} | {test_m['tpr']*100:<12.2f}")
print(f"{'FPR':<15} | {train_m['fpr']*100:<12.2f} | {val_m['fpr']*100:<12.2f} | {test_m['fpr']*100:<12.2f}")
print(f"{'Loss':<15} | {train_m['loss']:<12.4f} | {val_m['loss']:<12.4f} | {test_m['loss']:<12.4f}")
print("\n" + "="*80)
print("✅ TRAINING COMPLETE")
print("="*80)

In [ ]:
# Save results
catboost_model.save_model('swin_base_hybrid_catboost.cbm')
np.savez('swin_base_features.npz', X_train=X_train, y_train=y_train, 
         X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test)

# Download
from google.colab import files
files.download('swin_base_hybrid_catboost.cbm')
files.download('swin_base_features.npz')

print("💾 Files saved and ready for download!")